In [ ]:
# Install required packages
!pip install --upgrade streamlit pyngrok pandas scikit-learn

  Using cached pyngrok-7.5.0-py3-none-any.whl.metadata (8.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.1 MB/s eta 0:00:00
Using cached pyngrok-7.5.0-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 112.1 MB/s eta 0:00:00
  Attempting uninstall: pyngrok
    Found existing installation: pyngrok 5.2.0
    Uninstalling pyngrok-5.2.0:
      Successfully uninstalled pyngrok-5.2.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependenc

In [ ]:
from google.colab import files

# Upload your trained model (fraud_model.pkl)
uploaded = files.upload()


Saving fraud_model.pkl to fraud_model (1).pkl


In [ ]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd

# Load trained Logistic Regression model
model = pickle.load(open("fraud_model.pkl", "rb"))

st.title("💳 Credit Card Fraud Detection")

input_method = st.radio("Choose input method:", ("Manual Input", "Upload CSV"))

if input_method == "Manual Input":
    time = st.number_input("Time")
    amount = st.number_input("Amount")
    features = {}
    for i in range(1, 29):
        features[f"V{i}"] = st.number_input(f"V{i}")
    if st.button("Predict"):
        input_data = pd.DataFrame([[time, amount] + list(features.values())],
                                  columns=['Time', 'Amount'] + [f'V{i}' for i in range(1,29)])
        prediction = model.predict(input_data)
        probability = model.predict_proba(input_data)[0][1]
        if prediction[0] == 1:
            st.error(f"🚨 Fraudulent Transaction! (Confidence: {probability:.2f})")
        else:
            st.success(f"✅ Legitimate Transaction (Confidence: {1 - probability:.2f})")
elif input_method == "Upload CSV":
    uploaded_file = st.file_uploader("Upload CSV with columns Time, Amount, V1–V28", type=['csv'])
    if uploaded_file is not None:
        data = pd.read_csv(uploaded_file)
        if all(col in data.columns for col in ['Time','Amount'] + [f'V{i}' for i in range(1,29)]):
            predictions = model.predict(data)
            probabilities = model.predict_proba(data)[:,1]
            data['Prediction'] = ['Fraud' if p==1 else 'Legit' for p in predictions]
            data['Fraud_Probability'] = probabilities
            st.dataframe(data)
            csv = data.to_csv(index=False).encode('utf-8')
            st.download_button("Download Predictions", data=csv, file_name='predictions.csv', mime='text/csv')
        else:
            st.error("❌ CSV missing required columns.")


Overwriting app.py


In [ ]:
!ngrok authtoken 37extqEaHm11VZPnNBrl90cuv6R_7NVSMULE8vCX4W8xBxMjm


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [1]:
from pyngrok import ngrok
import os

# Kill any previous tunnels
ngrok.kill()

# Ensure the latest ngrok agent is installed (latest pyngrok should handle this)
ngrok.install_ngrok()

# Run Streamlit in the background
os.system("streamlit run app.py &")

# Open a tunnel to port 8501 (default Streamlit port)
public_url = ngrok.connect(8501)
print("Streamlit app is live at:", public_url)

Streamlit app is live at: NgrokTunnel: "https://subhemispheric-sara-triunitarian.ngrok-free.dev" -> "http://localhost:8501"
